<center> <h2> DS 3000 - Spring 2019</h2> </center>
<center> <h3> DS Report </h3> </center>


<center> <h3> Influence of US Demographic Factors on Personal Income </h3> </center>
<center><h4>Aaditya Watwe, Arian Gokhale, Lily Olson, Alex Shaw</h4></center>


<hr style="height:2px; border:none; color:black; background-color:black;">

#### Executive Summary:

Add your summary here (100-150 words)

Provide a brief summary of your project. After reading this executive summary, your readers should have a rough understanding of what you did in this project. You can think of this summary in terms of the four sections of the report and write 1-2 sentences describing each section.



<hr style="height:2px; border:none; color:black; background-color:black;">

## Outline
1. <a href='#1'>INTRODUCTION</a>
2. <a href='#2'>METHOD</a>
3. <a href='#3'>RESULTS</a>
4. <a href='#4'>DISCUSSION</a>

<a id="1"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 1. INTRODUCTION

Our project decided to tackle understanding how different demographics can affect the personal income of individuals. We believe that there are some data visualizations tackling this issue, but many are highly politicized or hard to interpet. This has led to a lack of understanding on this topic by the general pulic. The topic of our project is determining to what degree an income gap exists and along which demographic borders does it fall. Some of our learning objectives include analyzing the gender-based income gap and the model minority theories. We would also like to analyze which demographic factors are the most significant in influencing one's income.

This is an important problem to tackle, because without an unbiased understanding of the problem, we cannot create well informed public policy. The lack of concrete evidence towards either side of the political divide over this issue creates more misinformation and angst rather than approaching a solution. The insights derived from this project could show where the primary causes of income gaps are. Thus the problem is no longer the “income gap”, rather the problem will more clearly be identified (i.e. If the primary gap found is the lack of X race in Y industry, future studies could examine the cause of that phenomenon). There has been some research done in this area. A report titled Income and Wealth in the United States by the Peter G. Peterson Foundation found that incomes varied considerably between the coasts and the midwest and the south of the United States. Furthermore, income varied based on race/ethnicity, educational attainment, and gender. However, the analysis fozues on the median income levels, and does not produce any correlataional analysis (1). Other projects have looked into how the middle class has shrunk over time in America, but not dived into specifics of demographic data (2). 

Given the aforementioned problem and its importance, we set out to tackle the following questions:
* To what degree do factors including gender, race, family wealth, geographic area, and occupational industry influence one's income?
* To what extent does the gender-based income gap exist and are there deviations for different wealth brackets, races, or geographic areas?

<a id="2"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 2. METHOD

### 2.1. Data Acquisition

* Describe where you obtained your data. Provide a link to the original source. 
* If you scraped your data, include your code as a separate script file.
* Your data should be stored in an online repository (e.g., GitHub) and your code should retrieve your data from that online resource. You can read csv files from the Web in the same way that you read files from local drive.
* Describe the dataset and variables. What do variables represent?


### 2.2. Variables
* For your hypotheses, what are your IVs and DVs?
* For your predictive models, what are your features and target variables?


### 2.3. Data Analysis
* Specifically describe your predictive model. What outcome variable are you going to predict from what feature variables?
* Describe whether this is a supervised or unsupervised learning problem. Also identify the sub-category of the learning task (e.g. classification).
* What machine learning algorithms are you going to use? Why?

<a id="3"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 3. RESULTS

### 3.1. Data Wrangling
* Perform simple data cleaning (delete extra columns, deal with NA values, etc.)
* Perform data wrangling to extract your features and target values (e.g., grouping your dataframe by columns, applying functions to format dataframes, etc.)
* Preprocess your variables (e.g., scaling/transforming feature variables to normalize them)
* Feature extraction (dummy variables, new features from existing features, etc.)
* Use one feature selection technique to select a subset of your original features


### 3.2. Data Exploration
* Generate appropriate data visualizations for your key variables identified in the previous section
* You should have at least three visualizations (and at least two different visualization types)
* For each visualization provide an explanation regarding the variables involved and an interpretation of the graph.
* If you are using Plotly, insert your visualizations as images as well (upload the graph images to an online source, e.g. github, and link those in Jupyter Notebook)


### 3.3. Model Construction
* If you proposed hypotheses, conduct your hypothesis tests
* For your machine learning question(s), split data into training, validation, and testing sets (or use cross-validation)
* Apply machine learning algorithms (apply at least three algorithms)
* Train your algorithms

### 3.4. Model Evaluation
* Evaluate the performance of your algorithms on appropriate evaluation metrics, using your validation set
* Interpret your results from multiple models (and hypothesis tests, if any)

### 3.5. Model Optimization
* Tune your models using appropriate hyperparameters
* Explain why you are doing this (e.g., to avoid overfitting, etc.)

### 3.6. Model Testing
* Test your tuned algorithms using your testing set

<a id="4"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 4. DISCUSSION
* Provide a summary of the steps you took to analyze your data and test your predictive model
* Intepret your findings from 3.4., 3.5, and 3.6
    * Which algorithms did you compare?
    * Which algorithm(s) revealed best performance?
    * Which algorithm(s) should be used for your predictive model?
* If you tested hypotheses, interpret the results. What does it mean to have significant/non-significant differences with regards to your data?


* End this section with a conclusion paragraph containing some pointers for future work 
    *(e.g., get more data, perform another analysis, etc.)

<a id="5"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

### CONTRIBUTIONS
* Describe each team member's contributions to the report (who did what in each section)
* Remember this is a team effort!
* Each member of your team will provide peer evaluation of other team members. Your final grade on the project will be based on those peer evaluations. 